# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,8.38,97,100,0.60,SJ,1721587139
1,1,grytviken,-54.2811,-36.5092,0.95,88,66,4.62,GS,1721587140
2,2,san luis,-33.2950,-66.3356,13.85,51,95,3.36,AR,1721586902
3,3,toliara,-23.3500,43.6667,20.14,79,0,4.61,MG,1721587143
4,4,punta arenas,-53.1500,-70.9167,7.84,75,100,12.08,CL,1721587144


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat', 
    geo=True, 
    size='Humidity', 
    color='Humidity', 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600,
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_data_df = city_data_df[(city_data_df["Max Temp"] < 32) & (city_data_df["Humidity"] < 70)]
# Drop any rows with null values
narrowed_city_data_df = narrowed_city_data_df.dropna()

# Display sample data
narrowed_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,san luis,-33.2950,-66.3356,13.85,51,95,3.36,AR,1721586902
5,5,adamstown,-25.0660,-130.1015,18.80,66,95,8.88,PN,1721587145
15,15,carnarvon,-24.8667,113.6333,18.62,51,0,1.30,AU,1721587156
18,18,bethel,41.3712,-73.4140,30.52,54,100,1.34,US,1721587135
20,20,pacasmayo,-7.4006,-79.5714,19.89,68,4,7.42,PE,1721587162
...,...,...,...,...,...,...,...,...,...,...
567,567,tucuma,-0.1833,-50.0500,31.83,54,14,4.45,BR,1721587791
568,568,vale da amoreira,39.0697,-7.6985,29.09,29,0,6.23,PT,1721587814
573,573,monmouth,40.3001,-74.1663,31.87,60,100,4.50,US,1721587821
574,574,yorkton,51.2167,-102.4677,29.12,50,22,2.20,CA,1721587823


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,san luis,AR,-33.2950,-66.3356,51,
5,adamstown,PN,-25.0660,-130.1015,66,
15,carnarvon,AU,-24.8667,113.6333,51,
18,bethel,US,41.3712,-73.4140,54,
20,pacasmayo,PE,-7.4006,-79.5714,68,
...,...,...,...,...,...,...
567,tucuma,BR,-0.1833,-50.0500,54,
568,vale da amoreira,PT,39.0697,-7.6985,29,
573,monmouth,US,40.3001,-74.1663,60,
574,yorkton,CA,51.2167,-102.4677,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
latitude = hotel_df["Lat"]
longitude = hotel_df["Lng"]
categories = "accommodation.hotel"
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san luis - nearest hotel: Hotel Dos Venados
adamstown - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
pacasmayo - nearest hotel: Playa Linda
edinburgh of the seven seas - nearest hotel: No hotel found
jwaneng - nearest hotel: Mokala Lodge & Teemane Casino
albany - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
strathdale - nearest hotel: All Seasons Resort Hotel Bendigo
columbus - nearest hotel: Sheraton Columbus Hotel at Capitol Square
iqaluit - nearest hotel: Frobisher Inn
fale old settlement - nearest hotel: No hotel found
conceicao das alagoas - nearest hotel: No hotel found
burela de cabo - nearest hotel: Casa do Mudo
yellowknife - nearest hotel: No hotel found
khanty-mansiysk - nearest hotel: Гамма
badger - nearest hotel: Clarion Hotel & Suites
usogorsk - nearest hotel: Единственная
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
polyarnyy - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
2,san luis,AR,-33.2950,-66.3356,51,Hotel Dos Venados
5,adamstown,PN,-25.0660,-130.1015,66,No hotel found
15,carnarvon,AU,-24.8667,113.6333,51,No hotel found
18,bethel,US,41.3712,-73.4140,54,Hampton Inn Danbury
20,pacasmayo,PE,-7.4006,-79.5714,68,Playa Linda
...,...,...,...,...,...,...
567,tucuma,BR,-0.1833,-50.0500,54,No hotel found
568,vale da amoreira,PT,39.0697,-7.6985,29,No hotel found
573,monmouth,US,40.3001,-74.1663,60,Extended Stay America - Red Bank - Middletown
574,yorkton,CA,51.2167,-102.4677,50,Holiday Inn Express


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# hotel_df = hotel_df.dropna()

In [20]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    size='Humidity', 
    color='Country', 
    tiles='OSM', 
    frame_width=800, 
    frame_height=600,
        hover_cols=['Country', 'Hotel Name']
)
# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name)